### Обзор алгоритмов рекомендательных систем

Несмотря на множество существующих алгоритмов, все они сводятся к нескольким базовым подходам, которые будут описаны далее. К наиболее классическим относятся алгоритмы Summary-based (неперсональные), Content-based (модели основанные на описании товара), Matrix Factorization (методы основанные на матричном разложении), Collaborative Filtering (коллаборативная фильтрация) и гибридные.

#### Неперсонализированные рекомендации (summary-based)

Начнем с неперсонализированных рекомендаций, поскольку они самые простые в реализации. В них потенциальный интерес пользователя определяется просто средним рейтингом.


##### Проблема холодного старта

Холодный старт – это типичная ситуация, когда ещё не накоплено достаточное количество данных для корректной работы рекомендательной системы (например, когда товар новый или просто его очень редко покупают). Если средний рейтинг посчитан по оценкам всего трёх пользователей, такая оценка явно не будет достоверной, и пользователи это понимают. Часто в таких ситуациях рейтинги искусственно корректируют.

##### Актуальность рекомендаций

В некоторых случаях также важно учитывать «свежесть» рекомендации. Это особенно актуально для статей или постов на форумах. Пример расчета рейтинга в журнале Hacker news:
![Untitled.jpg](https://hsto.org/r/w1560/webt/vc/yr/sn/vcyrsnoylizvctaq3ilbafaosbq.jpeg)

где U = upvotes, D = downvotes, а P (Penalty) — дополнительная корректировка для имплементации иных бизнес-правил

Расчет рейтинга в Reddit:
![image.png](https://hsto.org/r/w1560/webt/g-/zj/fg/g-zjfghqsstjvuryzd-b9c92hjk.jpeg)

где U = число голосов «за», D = число голосов «против», T = время записи. Первое слагаемое оценивает «качество записи», а второе делает поправку на время.

#### Content-based рекомендации
Персональные рекомендации предполагают максимальное использование информации о самом пользователе, в первую очередь о его предыдущих покупках. Товары и услуги рекомендуются на основе знаний о них. Одним из первых появился подход content-based filtering.


![image.png](https://hsto.org/r/w1560/webt/ns/yf/-l/nsyf-lljlfb0r7bjrc_l3xonto0.jpeg)

В качестве меры близости двух векторов чаще всего используется косинусное расстояние.
![0a6136c9bd4144fabc84b53b4e0b6f49.png](https://hsto.org/r/w1560/getpro/habr/post_images/0a6/136/c9b/0a6136c9bd4144fabc84b53b4e0b6f49.png)

In [1]:
# Loading the data
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
ds = pd.read_csv('https://raw.githubusercontent.com/alex-coch/Recommender-systems-review/main/sample-data.csv')

# Creating a TF-IDF Vectirizer and calculate the TF-IDF score for each document's description
# tfidt_matrix is the matrix containing each word and its TF-IDF score
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(ds['description'])

# Calculating Cosine Similarity of each item with every other item in the dataset, and the arranging
# them according to their similarity with item i, and storing values in results
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
results = {}
for idx, row in ds.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], ds['id'][i]) for i in similar_indices]

    results[row['id']] = similar_items[1:]
print('done!')

# Making a recommendation
def item(id):
    return ds.loc[ds['id'] == id]['description'].tolist()[0].split(' - ')[0]


# Just reads the results out of the dictionary.
# The function collects the results[] corresponding to that item_id
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to '" + item(item_id) + "'...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")


recommend(item_id=11, num=5)

done!
Recommending 5 products similar to 'Baby sunshade top'...
-------
Recommended: Sunshade hoody (score:0.21330296021085024)
Recommended: Baby baggies apron dress (score:0.10975311296284812)
Recommended: Runshade t-shirt (score:0.09988151262780731)
Recommended: Runshade t-shirt (score:0.09530698241688207)
Recommended: Runshade top (score:0.08510550093018411)


#### Алгоритмы факторизации (Matrix Factorization)
Было бы здорово описать интересы пользователя более «крупными мазками». Не в формате «он любит фильмы X, Y и Z», а в формате «он любит современные российские комедии». Помимо того, что это увеличит обобщаемость модели, это еще решит проблему большой размерности данных — ведь интересы будут описываться не вектором товаров, а существенно меньшим вектором предпочтений.

Такие подходы еще называют спектральным разложением или высокочастотной фильтрацией (поскольку мы убираем шум и оставляем полезный сигнал). В алгебре существует много различных разложений матриц, и одно из наиболее часто используемых называется SVD-разложением (singular value decomposition). В основе метода — разложение исходной матрицы рейтингов ® в произведение 3 матриц:

![2c83c9208eb951771aefef402c22b6f2.svg](https://hsto.org/getpro/habr/formulas/2c8/3c9/208/2c83c9208eb951771aefef402c22b6f2.svg), где размеры матриц ![dd9b62bd17c350ddcc2771beb9f6c400.svg](https://hsto.org/getpro/habr/formulas/dd9/b62/bd1/dd9b62bd17c350ddcc2771beb9f6c400.svg)

Применяя данное разложение к нашей матрице предпочтений, мы получаем две матрицы факторов (сокращенных описаний):
U — компактное описание предпочтений пользователя,
S — компактное описание характеристик продукта.

Для того, чтобы получить приближение матрицы предпочтений достаточно перемножить матрицы факторов. Сделав это получим оценку рейтинга для всех пар клиент-продукт.

[Пример реализации алгоритма Matrix Factorization](https://colab.research.google.com/drive/1uCK_qUqrGQ_MYGIisF_xpuzarjEHMGJy)

#### Collaborative Filtering (коллаборативная фильтрация)
В рамках подхода рекомендации генерируются на основании интересов других похожих пользователей (User-based вариант). Такие рекомендации являются результатом «коллаборации» множества пользователей.
Классическая реализация алгоритма основана на принципе k ближайших соседей, где для каждого пользователя ищем k наиболее похожих на него (в терминах предпочтений) и дополняем информацию о пользователе известными данными по его соседям.
![image.png](https://hsto.org/r/w1560/webt/al/it/fv/alitfv0yszejlvhji0agoy01amg.jpeg)

На картинке выше проиллюстрирован принцип работы метода. В матрице предпочтений желтым цветом выделен пользователь, для которого мы хотим определить оценки по новым товарам (знаки вопроса). Синим цветом выделены три его ближайших соседа.


##### Стандартизация данных (scaling)
Поскольку все пользователи оценивают по-разному – кто-то всем подряд пятерки ставит, а от кого-то четверки редко дождешься – перед расчетом данные лучше нормализовать, т.е. привести к единой шкале, чтобы алгоритм мог корректно сравнивать их между собой.

Подход Item-based является естественной альтернативой классическому подходу User-based, описанному выше, и почти полностью его повторяет, за исключением одного момента — применяется он к транспонированной матрице предпочтений. Т.е. ищет близкие товары, а не пользователей.

[Пример реализации алгоритма Collaborative Filtering](https://colab.research.google.com/drive/1X4QC7-sR3f2P7jvxinQXDeTEUwu0SUJC)

#### Гибридные решения (hybrid)

Есть несколько распространенных типов комбинирования:
*   реализация по отдельности коллаборативных и контентных алгоритмов и объединение их предположений;
*   включение некоторых контентных правил в коллаборативную методику;
*   включение некоторых коллаборативных правил в контентную методику;
*   построение общей модели, включающей в себя правила обеих методик.




Несколько стратегий объединения:
*   Weighting — считать средневзвешенный прогноз по нескольким оценкам;
*   Stacking — предсказания отдельных моделей являются входами другого (мета)классификатора, который обучается правильно взвешивать промежуточные оценки;
*   Switching — для разных продуктов/пользователей применять различные алгоритмы;
*   Mixing — вычисляются рекомендации по разным алгоритмам, а потом просто объединяются в один список.

Гибридные модели рекомендации:
*   Машина Факторизации (Factorization Machine);
*   Широкие и глубокие: нейронная коллаборативная фильтрация (NCF) и Глубокие Машины Факторизации (DeepFM);
*   DLRM – рекомендационная модель глубокого обучения. 


#### Другие подходы

*   Ассоциативные правила (Association Rules) - если мы видим, что молоко в корзину клиент уже положил, самое время напомнить о хлебе;
*   RBM (restricted Bolzman Machines) - ищется наиболее компактное описание пользовательских предпочтений;
*   Автоэнкодеры (autoencoders) - получается некий усредненный, очищенный от шума шаблон (данные о пользователе), по которому можно оценить интерес к любому продукту;
*   DSSM (deep sematic similiarity models) - в роли латентных переменных здесь внутренние тензорные описания входных данных (embeddings).


Источники данных:


*   https://habr.com/ru/company/lanit/blog/420499/ 
*   https://habr.com/ru/company/lanit/blog/421401/
*   https://grouplens.org/datasets/movielens/
*   https://heartbeat.fritz.ai/recommender-systems-with-python-part-i-content-based-filtering-5df4940bd831
*   https://heartbeat.fritz.ai/recommender-systems-with-python-part-ii-collaborative-filtering-k-nearest-neighbors-algorithm-c8dcd5fd89b2 
*   https://heartbeat.fritz.ai/recommender-systems-with-python-part-iii-collaborative-filtering-singular-value-decomposition-5b5dcb3f242b



